In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nibabel as nib
import matplotlib.pyplot as plt
from dataset import CarotidDataset
from unet import UNet

from torchvision import transforms

In [2]:
class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.input_batchnorm = nn.BatchNorm2d(kwargs['in_channels'])
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()
        #self._init_weights()
    
    def forward(self, input_batch):
        bn_output = self.input_batchnorm(input_batch)
        un_output = self.unet(bn_output)
        fn_output = self.final(un_output)
        return fn_output

In [3]:
data = CarotidDataset()

epochs = 2
BATCH_SIZE = 4
#net = UNet(in_channels=3, n_classes=1, padding=True, up_mode='upsample')
net = UNetWrapper(in_channels=3, n_classes=1, padding=True, up_mode='upsample')
device = torch.device("mps")
#device = torch.device("cpu")
trainloader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
step_count = 0
loss = nn.CrossEntropyLoss()

#transforms = torch.nn.Sequential(
#    transforms.CenterCrop((736, 704))
#)

transforms = torch.nn.Sequential(
    transforms.CenterCrop(512)
)

In [4]:
net.train()
net = net.to(device)
train_running_loss = 0.0
train_running_corrects = 0

In [5]:
inputs, labels = next(iter(trainloader))
inputs = inputs.to(device)
labels = labels.to(device)
optimizer.zero_grad()
preds = net(inputs)
#preds = F.pad(preds, (3,2,7,6))

In [ ]:
transforms(inputs).shape

In [6]:
preds.shape

torch.Size([4, 1, 512, 512])

In [7]:
inputs.shape

torch.Size([4, 3, 512, 512])

In [8]:
labels.shape

torch.Size([4, 1, 512, 512])